Name: Xinnan Shen <br>
Student ID: 1051380

This notebook is used for building machine learning models and use the model to train and make predictions.
<br>
**Note: The notebook should be run on Google Colab if possible.**

First, we need to upload datasets to colab.<br>
These datasets include training, development and testing dataset.<br>
The training dataset has already been expanded by crawling some additional data from website. (crawl.py and clean.py)

In [1]:
from google.colab import files

print("upload training data (Note: File name must be 'newtrain.json')")
train_uploaded = files.upload()
print("upload development data (Note: File name must be 'dev.json')")
dev_uploaded = files.upload()
print("upload testing data (Note: File name must be 'test-unlabelled.json')")
test_uploaded = files.upload()


upload training data (Note: File name must be 'newtrain.json')


Saving newtrain.json to newtrain.json
upload development data (Note: File name must be 'dev.json')


Saving dev.json to dev.json
upload testing data (Note: File name must be 'test-unlabelled.json')


Saving test-unlabelled.json to test-unlabelled.json


Next, we need to preprocess the dataset bedore building the model.<br>
The preprocessing step includes:<br>
(1)tokenize text<br>
(2)lemmatize words<br>
(3)remove stopwords and unwanted words

In [2]:
import os
import codecs
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
def get_pos(word):
    tag=nltk.tag.pos_tag([word])[0][1]
    if tag[0]=='N':
        return wordnet.NOUN
    elif tag[0]=='V':
        return wordnet.VERB
    elif tag[0]=='J':
        return wordnet.ADJ
    elif tag[0]=='R':
        return wordnet.ADV
    else:
        return wordnet.NOUN
def preprocessing(str):
	s=str
	tt = TweetTokenizer()
	stopwordslist = set(stopwords.words('english'))
	lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
	dict=eval(s)
	out_dict={}
	for id,con in dict.items():
		print("Processing-"+id)
		text=con["text"]
		li=tt.tokenize(text)
		token_list=[]
		for ele in li:
			ele=ele.lower()
			if re.search(r"[a-zA-Z]+",ele):
				pos=get_pos(ele)
				token_list.append(lemmatizer.lemmatize(ele,pos).lower())
				
		result_dict={}
		for t in token_list:
			if t not in stopwordslist:
				if t not in result_dict:
					result_dict[t]=1
				else:
					result_dict[t]=result_dict[t]+1
		con["text"]=result_dict
		out_dict[id]=con
	return out_dict
def readfile(filename):
	current_path=os.path.abspath(os.curdir)
	file_path=os.path.join(current_path,filename)
	if not os.path.exists(file_path):
		print("error:file not found:"+filename)
		return ""
	f=codecs.open(file_path,"r","utf-8")
	s=f.read()
	f.close()
	return s
train_data=readfile('newtrain.json')#train_uploaded['newtrain.json'].decode("utf-8")
dev_data=readfile('dev.json')#dev_uploaded['dev.json'].decode("utf-8")
test_data=readfile('test-unlabelled.json')#test_uploaded['test-unlabelled.json'].decode("utf-8")
dict_train=preprocessing(train_data)
dict_dev=preprocessing(dev_data)
dict_test=preprocessing(test_data)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Processing-train-0
Processing-train-1
Processing-train-2
Processing-train-3
Processing-train-4
Processing-train-5
Processing-train-6
Processing-train-7
Processing-train-8
Processing-train-9
Processing-train-10
Processing-train-11
Processing-train-12
Processing-train-13
Processing-train-14
Processing-train-15
Processing-train-16
Processing-train-17
Processing-train-18
Processing-train-19
Processing-train-20
Processing-train-21
Processing-train-22
Processing-train-23
Processing-train-24
Processing-train-25
Processing-train-26
Processing-train-27
Processing-train-28
Processi

After completing preprocessing, we should also convert the bag-of-words representation into feature vectors to make it easier to build and train the model.

In [3]:
from sklearn.feature_extraction import DictVectorizer
import numpy as np
data_train=[]
Y_train=[]
for id,con in dict_train.items():
	data_train.append(con["text"])
	Y_train.append(con["label"])
data_dev=[]
Y_dev=[]
for id,con in dict_dev.items():
	data_dev.append(con["text"])
	Y_dev.append(con["label"])
data_test=[]
for id,con in dict_test.items():
	data_test.append(con["text"])
vectorizer = DictVectorizer(sparse=False)
X_train=vectorizer.fit_transform(data_train)
X_dev=vectorizer.transform(data_dev)
X_test=vectorizer.transform(data_test)
temp=[]
for i in Y_train:
	temp.append([i])
Y_train=np.array(temp)
temp=[]
for i in Y_dev:
	temp.append([i])
Y_dev=np.array(temp)
print(X_train.shape,X_dev.shape,X_test.shape,Y_train.shape,Y_dev.shape)

(3432, 46264) (100, 46264) (1410, 46264) (3432, 1) (100, 1)


Before we using nerual network, we need to first analyse the performance of traditional models.<br>
Take Naive Bayes as an example.

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
score=0
set_alpha=-1
#define model
model=MultinomialNB()
#hyper-parameter tuning
for a in [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000]:
    model.set_params(alpha=a).fit(X_train,Y_train)
    predictions=model.predict(X_dev)
    this_score=accuracy_score(Y_dev,predictions)
    print("alpha= ",a,",score=",round(this_score,3))
    if this_score>score:
        score=this_score
        set_alpha=a
#get the best hyper-parameter
model.set_params(alpha=set_alpha).fit(X_train,Y_train)
print("Best Score:",round(score,3),",where alpha=",set_alpha)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


alpha=  1e-05 ,score= 0.76


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


alpha=  0.0001 ,score= 0.75


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

alpha=  1000 ,score= 0.58


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


alpha=  10000 ,score= 0.59


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Score: 0.76 ,where alpha= 1e-05


In [5]:
#make predictions of the NB model
import numpy as np

Y_dev_res=model.predict(X_dev)

result=[]
for y in np.nditer(Y_dev_res):
  result.append(round(float(y)))
i=0
result_data={}
for key in eval(dev_data).keys():
  result_data[key]={"label":result[i]}
  i=i+1
f=codecs.open("dev-output.json","w","utf-8")
f.write(str(result_data).replace("'",'"'))
f.close()

In [6]:
files.download("dev-output.json")#download the result file

Then, we can build the nerual network and tune the hyper-parameters of the model.

In [7]:
import tensorflow as tf
from tensorboard.plugins.hparams import api
from keras import models as md
from keras import layers as lr
from keras.layers import LSTM
#save the best model, its accuracy and hyper-parameters
best_model=""
best_acc=0.0
best_hp={}
#set some parameters for use
units1=api.HParam("num_units_layer_1",api.Discrete([4,8,16,32,64,128]))#dense layer
units2=api.HParam("num_units_layer_2",api.Discrete([4,8,16,32,64,128]))#dense layer
units3=api.HParam("num_units_layer_2",api.Discrete([4,8,16,32,64,128]))#dense layer
optimizerfunc=api.HParam("optimizer",api.Discrete(["adam","sgd"]))#optimizer
#develop the model
def dev_model(p):
	global best_acc
	global best_model
	global best_hp
	#define the model
	model = md.Sequential()
	model.add(lr.Dense(p[units1],activation="tanh"))
	model.add(lr.Dense(p[units2],activation="tanh"))
	model.add(lr.Dense(p[units3],activation="tanh"))
	model.add(lr.Dense(1,activation="sigmoid"))
	model.compile(optimizer=p[optimizerfunc],loss="binary_crossentropy",metrics=["accuracy"])#compile the model
	model.fit(X_train, Y_train, epochs=3, batch_size=3)#fit the model
	loss,acc = model.evaluate(X_dev, Y_dev)#record loss and accuracy
	print("loss: ",loss,", accuracy=",acc)
	#update the best model and its accuracy
	if acc>best_acc:
		best_model=model
		best_acc=acc
		best_hp={units1:p[units1],units2:p[units2],units3:p[units3],optimizerfunc:p[optimizerfunc]}
	return
#iteratively changing the hyper-parameters and record accuracy
for num_units_layer_1 in units1.domain.values:
	for num_units_layer_2 in units2.domain.values:
		for num_units_layer_3 in units3.domain.values:
			for optimizer in optimizerfunc.domain.values:
				#print hyper-parameters
				print("num_units_layer_1:"+str(num_units_layer_1),"num_units_layer_2:"+str(num_units_layer_2),"num_units_layer_3:"+str(num_units_layer_3),"optimizer:"+str(optimizer))
				#using the hyper-parameter to develop the model
				dev_model({units1:num_units_layer_1, units2:num_units_layer_2, units3:num_units_layer_3, optimizerfunc:optimizer})
#tunning hyperparameter completed,set the best model
model=best_model
print(best_hp)

100/100 [==============================] - 0s 464us/step
loss:  0.883062653541565 , accuracy= 0.8100000023841858
num_units_layer_1:128 num_units_layer_2:128 num_units_layer_3:4 optimizer:sgd
Epoch 1/3
3432/3432 [==============================] - 9s 3ms/step - loss: 0.1589 - accuracy: 0.9595
Epoch 2/3
3432/3432 [==============================] - 8s 2ms/step - loss: 0.0590 - accuracy: 0.9863
Epoch 3/3
100/100 [==============================] - 0s 459us/step
loss:  0.5982247769832612 , accuracy= 0.8299999833106995
num_units_layer_1:128 num_units_layer_2:128 num_units_layer_3:8 optimizer:adam
Epoch 1/3
3432/3432 [==============================] - 16s 5ms/step - loss: 0.0902 - accuracy: 0.9714
Epoch 2/3
3432/3432 [==============================] - 17s 5ms/step - loss: 0.0082 - accuracy: 0.9994
Epoch 3/3
100/100 [==============================] - 0s 470us/step
loss:  0.8768482112884521 , accuracy= 0.8299999833106995
num_units_layer_1:128 num_units_layer_2:128 num_units_layer_3:8 optimizer:sg

This step is for evaluation only.<br>
We evaluate our model on development set.

In [8]:
import numpy as np

Y_dev_res=model.predict(X_dev)

result=[]
for y in np.nditer(Y_dev_res):
  result.append(round(float(y)))
i=0
result_data={}
for key in eval(dev_data).keys():
  result_data[key]={"label":result[i]}
  i=i+1
f=codecs.open("dev-output.json","w","utf-8")
f.write(str(result_data).replace("'",'"'))
f.close()


After evaluation, we download the result (dev-output.json) file.

In [9]:
files.download("dev-output.json")

Finally, we can make predictions and get the test results based on previous model.

In [10]:
import numpy as np

Y_test=model.predict(X_test)

result=[]
for y in np.nditer(Y_test):
  result.append(round(float(y)))
i=0
result_data={}
for key in eval(test_data).keys():
  result_data[key]={"label":result[i]}
  i=i+1
f=codecs.open("test-output.json","w","utf-8")
f.write(str(result_data).replace("'",'"'))
f.close()


After getting the result, we can download result file (test-output.json).

In [11]:
files.download("test-output.json")